In [1]:
import os
import sys

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)


In [2]:
import tensorflow as tf
from tensorflow import keras
from keras import layers

# Load MNIST
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Preprocess the data by flattening & scaling it
x_train = x_train.reshape(-1, 784).astype("float32") / 255
x_test = x_test.reshape(-1, 784).astype("float32") / 255

# Categorical (one hot) encoding of the labels
y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)


In [3]:
def _tf_ann(X, samples, p=2, soft=True):
    m_dis = None
    for i in range(samples.shape[0]):
        i_sample = samples[i : i + 1, :]
        l_dis = tf.math.reduce_sum((X - i_sample) ** p, axis=1, keepdims=True) ** (
            1 / p
        )
        if m_dis is None:
            m_dis = l_dis
        else:
            m_dis = tf.concat([m_dis, l_dis], 1)

    if soft:
        feature_map = tf.nn.softmax(-m_dis, axis=0)
    else:
        feature_map = tf.one_hot(tf.math.argmax(-m_dis, axis=1), samples.shape[0])
    # l_dis_min = tf.math.reduce_sum(m_dis * feature_map, axis=0)
    return feature_map


In [4]:
import os
import sys

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

# Load MNIST
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Preprocess the data by flattening & scaling it
x_train = x_train.reshape(-1, 784).astype("float32") / 255
x_test = x_test.reshape(-1, 784).astype("float32") / 255

# Categorical (one hot) encoding of the labels
y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)


In [9]:
def gen_samples(X, psi, t=1000):
    n, _ = X.shape
    return [
        tf.gather_nd(
            indices=tf.expand_dims(tf.random.shuffle(tf.range(n))[:psi], 1), params=X
        )
        for _ in range(t)
    ]


t_samples = gen_samples(x_train, psi=16, t=500)


In [6]:
class IsolationEncodingLayer(tf.keras.layers.Layer):
    def __init__(self, samples, p=2, soft=True, **kwargs):
      super(IsolationEncodingLayer, self).__init__(**kwargs)
      self.samples = samples
      self.p = p
      self.soft = soft

    def call(self, inputs):
      return _tf_ann(inputs, self.samples, self.p, self.soft)

In [10]:
def build_model(t_samples, p=2, soft=True):
    t = len(t_samples)
    if t <= 0:
        raise ValueError("t <= 0")
    _, dims = t_samples[0].shape

    inputs = keras.Input(name="inputs_x", shape=(dims,))
    lambdas = [
        IsolationEncodingLayer(t_samples[i], name="ann_{}".format(i))(inputs)
        for i in range(t)
    ]
    concatenated = layers.Concatenate(axis=1, name="concatenated")(lambdas)
    outputs = layers.Dense(units=10, name="outputs_y")(concatenated)

    model = keras.Model(name="isolation_encoding", inputs=inputs, outputs=outputs)
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=1e-3),
        loss=keras.losses.hinge,
        metrics=[keras.metrics.CategoricalAccuracy(name="acc")],
    )
    return model


model = build_model(t_samples)
model.summary()


Model: "isolation_encoding"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_x (InputLayer)           [(None, 784)]        0           []                               
                                                                                                  
 ann_0 (IsolationEncodingLayer)  (None, 16)          0           ['input_x[0][0]']                
                                                                                                  
 ann_1 (IsolationEncodingLayer)  (None, 16)          0           ['input_x[0][0]']                
                                                                                                  
 ann_2 (IsolationEncodingLayer)  (None, 16)          0           ['input_x[0][0]']                
                                                                                 

In [11]:
model.fit(x_train, y_train, epochs=20, batch_size=128, validation_split=0.2)


Epoch 1/20
375/375 [==============================] - 487s 850ms/step - loss: 0.1448 - acc: 0.8342 - val_loss: 0.0702 - val_acc: 0.9215
Epoch 2/20
375/375 [==============================] - 209s 557ms/step - loss: 0.0599 - acc: 0.9210 - val_loss: 0.0510 - val_acc: 0.9368
Epoch 3/20
375/375 [==============================] - 210s 561ms/step - loss: 0.0477 - acc: 0.9325 - val_loss: 0.0435 - val_acc: 0.9419
Epoch 4/20
375/375 [==============================] - 210s 560ms/step - loss: 0.0418 - acc: 0.9392 - val_loss: 0.0389 - val_acc: 0.9467
Epoch 5/20
375/375 [==============================] - 209s 558ms/step - loss: 0.0378 - acc: 0.9444 - val_loss: 0.0359 - val_acc: 0.9500
Epoch 6/20
375/375 [==============================] - 209s 559ms/step - loss: 0.0352 - acc: 0.9473 - val_loss: 0.0338 - val_acc: 0.9522
Epoch 7/20
375/375 [==============================] - 211s 563ms/step - loss: 0.0328 - acc: 0.9511 - val_loss: 0.0321 - val_acc: 0.9539
Epoch 8/20
375/375 [============================